In [1]:
import re
import torch
import matplotlib.pyplot as plt
import networkx as nx
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

2024-10-25 10:22:06.106485: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-25 10:22:06.125154: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-25 10:22:06.131127: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-25 10:22:06.145252: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-25 10:22:07.132877: W tensorflow/compiler/tf2

In [2]:
model_id = "meta-llama/Meta-Llama-3-8B"
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [3]:
# Load the model with 4-bit quantization
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_4bit=True,  # Enable 4-bit quantization
    device_map="auto",  # Automatically place model layers on GPU
    torch_dtype=torch.float16
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
# Initialize the text-generation pipeline
text_generator = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer
)

In [5]:
# Function to generate text
def generate_text(prompt_text, model_name, max_length=1000, num_return_sequences=1):
    if model_name == "LLama 3":
        response = text_generator(prompt_text, max_length=max_length, num_return_sequences=num_return_sequences)
    else:
        print("No model execution provided")
    return response

In [6]:
def extract_test_output(response):
    if response and len(response) > 0:
        generated_text = response[0].get('generated_text', '')
        # Extract the text after the last occurrence of "Test Output:"
        match = re.search(r'Test Output:\s*(.*)', generated_text, re.DOTALL)
        if match:
            return match.group(1).strip().split('\n')
    return []

In [7]:
# Function to parse model output
def parse_model_output(model_output):
    triples = []
    for output in model_output:
        lines = [line.strip() for line in output.strip().split('\n')]
        pattern = re.compile(r'(.+?)\s*\(([^,]+),\s*([^)]+)\)')
        for line in lines:
            matches = pattern.findall(line)
            for match in matches:
                relation, subject, obj = match
                subject = subject.strip()
                obj = obj.strip()
                triples.append({"sub": subject, "rel": relation, "obj": obj})
    return triples

In [8]:
# Function to process file and generate RDF triples
def process_file(file_path, model_name):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    prompt = f"""
    Here is the RDF data:
    {text}
    """
    model_response = generate_text(prompt, model_name, max_length=1000)
    test_output = extract_test_output(model_response)
    triples = parse_model_output(test_output)
    
    # Format triples for output
    formatted_triples = "\n".join([f"{triple['rel']}({triple['sub']}, {triple['obj']})" for triple in triples])
    
    # Call function to generate and save knowledge graph
    image_path = generate_knowledge_graph(triples)

    return formatted_triples, image_path  # Return formatted triples and image path

In [9]:
# Function to generate knowledge graph
def generate_knowledge_graph(rdf_triples):
    G = nx.DiGraph()
    for triple in rdf_triples:
        G.add_edge(triple['sub'], triple['obj'], relation=triple['rel'])

    pos = nx.spring_layout(G, seed=42, k=1.5)
    plt.figure(figsize=(14, 10))
    nx.draw_networkx_nodes(G, pos, node_size=2500, node_color='lightblue', edgecolors='black')
    nx.draw_networkx_edges(G, pos, edgelist=G.edges(), arrowstyle='->', arrowsize=30, edge_color='black')
    nx.draw_networkx_labels(G, pos, font_size=7, font_family='sans-serif', font_weight='bold')
    edge_labels = {(u, v): d['relation'] for u, v, d in G.edges(data=True)}
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='red', label_pos=0.5, font_size=8, rotate=True)
    plt.title('Knowledge Graph', fontsize=20, fontweight='bold')
    plt.axis('off')

    # Save the figure to a file
    image_path = "KG_images/knowledge_graph.png"  # Adjust the path if needed
    plt.savefig(image_path)
    plt.close()  # Close the plot to free memory
    return image_path

In [11]:
# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# RDF Triple Generator")
    gr.Markdown("Upload a text file, select a model, and generate RDF triples using LLama models.")

    # File upload component with filepath type
    file_input = gr.File(label="Upload Input Text File", type="filepath")
    
    # Dropdown menu for model selection
    selected_model = gr.Dropdown(choices=["LLama 3", "Mistral"], label="Select Model")

    # Generate button to trigger RDF generation
    generate_button = gr.Button("Generate RDF Triples")
    
    # Textbox to display the extracted triples
    triples_output = gr.Textbox(label="Extracted RDF Triples", lines=10)
    
    # Image output component to display the generated knowledge graph
    graph_output = gr.Image(label="Knowledge Graph", type="filepath")

    # Connect the button to the processing function
    generate_button.click(fn=process_file, inputs=[file_input, selected_model], outputs=[triples_output, graph_output])

In [12]:
# Launch the Gradio interface
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://8460aa9516e170c36a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/upb/users/b/balram/profiles/unix/cs/.local/lib/python3.12/site-packages/bitsandbytes/nn/modules.py:435: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
